In [1]:
T = 8

# sentence = '65+43=108\n' + ' '*T
# sentence = '12+46=058\n' + ' '*T
sentence = '56+58=114\n' + ' '*T

input = sentence[:T]
input

'65+43=10'

In [2]:
vocabs = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '+', '=', '\n', ' ']

stoi = {s: i for i, s in enumerate(vocabs)}
itos = {i: s for i, s in enumerate(vocabs)}

stoi

{'0': 0,
 '1': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9,
 '+': 10,
 '=': 11,
 '\n': 12,
 ' ': 13}

In [3]:
[stoi[s] for s in input]

[6, 5, 10, 4, 3, 11, 1, 0]

In [4]:
import numpy as np
emb_dim = 14

a = .1
b = 5

W_emb = np.array([
    [a, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],  # 0
    [1, a, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],  # 1
    [2, 0, a, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],  # 2
    [3, 0, 0, a, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],  # 3
    [4, 0, 0, 0, a, 0, 0, 0, 0, 0, 0, 0, 0, 0,],  # 4
    [5, 0, 0, 0, 0, a, 0, 0, 0, 0, 0, 0, 0, 0,],  # 5
    [6, 0, 0, 0, 0, 0, a, 0, 0, 0, 0, 0, 0, 0,],  # 6
    [7, 0, 0, 0, 0, 0, 0, a, 0, 0, 0, 0, 0, 0,],  # 7
    [8, 0, 0, 0, 0, 0, 0, 0, a, 0, 0, 0, 0, 0,],  # 8
    [9, 0, 0, 0, 0, 0, 0, 0, 0, a, 0, 0, 0, 0,],  # 9
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, b, 0, 0, 0,],  # +
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, b, 0, 0,],  # =
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, b, 0,],  # \n
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, b,],  # <pad>
], dtype=float)

# np.random.seed(42)
# W_emb += np.random.uniform(-0.0001, 0.0001, (len(vocabs), emb_dim))

assert W_emb.shape == (len(vocabs), emb_dim)
W_emb.round()

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [3., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [4., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [5., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [6., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [7., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [8., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [9., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 5., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 5., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 5., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 5.]])

In [5]:
x = np.array([W_emb[stoi[token]] for token in input])

assert x.shape == (T, emb_dim)
x

array([[6. , 0. , 0. , 0. , 0. , 0. , 0.1, 0. , 0. , 0. , 0. , 0. , 0. ,
        0. ],
       [5. , 0. , 0. , 0. , 0. , 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 5. , 0. , 0. ,
        0. ],
       [4. , 0. , 0. , 0. , 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. ],
       [3. , 0. , 0. , 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 5. , 0. ,
        0. ],
       [1. , 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. ],
       [0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. ]])

In [6]:
head_count = 1
d_head = emb_dim
d_model = d_head * head_count
d_model

14

In [7]:
W_Q = np.zeros((emb_dim, d_model))
assert W_Q.shape == (emb_dim, d_model)

In [8]:
W_K = np.zeros((emb_dim, d_model))

assert W_K.shape == (emb_dim, d_model)

In [9]:
W_V = np.array([
    [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
    [0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
    [0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
    [0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
    [0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
    [0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0,],
    [0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0,],
    [0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0,],
    [0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0,],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0,],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0,],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0,],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,],
])

assert W_V.shape == (emb_dim, d_model)

In [10]:
Q = x @ W_Q
K = x @ W_K
V = x @ W_V

assert Q.shape == (T, d_model)
assert K.shape == (T, d_model)
assert V.shape == (T, d_model)

In [11]:
import torch

M = torch.triu(torch.ones(T, T) * float('-inf'), diagonal=1)
M = M.masked_fill(torch.isnan(M), 0).numpy()
M


array([[  0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
       [  0.,   0., -inf, -inf, -inf, -inf, -inf, -inf],
       [  0.,   0.,   0., -inf, -inf, -inf, -inf, -inf],
       [  0.,   0.,   0.,   0., -inf, -inf, -inf, -inf],
       [  0.,   0.,   0.,   0.,   0., -inf, -inf, -inf],
       [  0.,   0.,   0.,   0.,   0.,   0., -inf, -inf],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0., -inf],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.]], dtype=float32)

In [12]:
m = {}

m[5] = [1,              float('-inf'),  float('-inf'),  1,              float('-inf'),  float('-inf')]
m[6] = [1,              float('-inf'),  float('-inf'),  1,              float('-inf'),  float('-inf'),  float('-inf')]
m[7] = [float('-inf'),  1,              float('-inf'),  float('-inf'),  1,              float('-inf'),  float('-inf'),  float('-inf')]

for k, v in m.items():
    M[k] = v + [float('-inf')] * (T - len(v))

M

array([[  0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
       [  0.,   0., -inf, -inf, -inf, -inf, -inf, -inf],
       [  0.,   0.,   0., -inf, -inf, -inf, -inf, -inf],
       [  0.,   0.,   0.,   0., -inf, -inf, -inf, -inf],
       [  0.,   0.,   0.,   0.,   0., -inf, -inf, -inf],
       [  1., -inf, -inf,   1., -inf, -inf, -inf, -inf],
       [  1., -inf, -inf,   1., -inf, -inf, -inf, -inf],
       [-inf,   1., -inf, -inf,   1., -inf, -inf, -inf]], dtype=float32)

In [13]:
QK = (Q @ K.T) / np.sqrt(d_model)
weights = torch.softmax(torch.from_numpy(QK+M), dim=-1)
weights

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.0000, 0.0000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.0000, 0.0000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.5000, 0.0000, 0.0000, 0.5000, 0.0000, 0.0000, 0.0000]],
       dtype=torch.float64)

In [14]:
Attention = weights @ V
Attention # Caveat: LayerNorm is not applied.

tensor([[12.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.2000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [11.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.1000,  0.1000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 7.3333,  0.0000,  0.0000,  0.0000,  0.0000,  0.0667,  0.0667,  0.0000,
          0.0000,  0.0000,  3.3333,  0.0000,  0.0000,  0.0000],
        [ 7.5000,  0.0000,  0.0000,  0.0000,  0.0500,  0.0500,  0.0500,  0.0000,
          0.0000,  0.0000,  2.5000,  0.0000,  0.0000,  0.0000],
        [ 7.2000,  0.0000,  0.0000,  0.0400,  0.0400,  0.0400,  0.0400,  0.0000,
          0.0000,  0.0000,  2.0000,  0.0000,  0.0000,  0.0000],
        [10.0000,  0.0000,  0.0000,  0.0000,  0.1000,  0.0000,  0.1000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [10.0000,  0.0000,  0.0000,  0.0000,  0.1000,  0.0000,  0.1000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0

In [15]:
ff_dim = d_model
# ff_dim = d_model * 4

W_1 = np.array([
    [1, 1, 1, -1, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
], dtype=float)
b_1 = np.array([0, -9, -10, 0, 9, 10, 0, 0, 0, 0, 0, 0, 0, 0])

assert W_1.shape == (d_model, ff_dim)
assert b_1.shape == (ff_dim,)

W_2 = np.array([
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
], dtype=float)

# Not ordinary bias
b_2 = np.array([
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
    [-10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
    [0, -10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
], dtype=float)
assert W_2.shape == (ff_dim, d_model)
assert b_2.shape == (T, d_model)

FFN = torch.relu(torch.from_numpy(Attention.numpy() @ W_1 + b_1)) @ W_2 + b_2
assert FFN.shape == Attention.shape == (T, d_model)
FFN.round()

tensor([[12.,  3.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [11.,  2.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 7.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 8.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 7.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [10., -9.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 8.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]],
       dtype=torch.float64)

In [16]:
import numpy as np

def sigmoid(x, lamb=-10):
    bias = 9.5
    return 1. / (1. + np.exp(lamb*(-x+bias)))
# import matplotlib.pyplot as plt
# plt.plot(np.arange(-30, 30, 0.1), sigmoid(np.arange(-30, 30, 0.1)))
# plt.plot(np.arange(-30, 30, 0.1), sigmoid(np.arange(-30, 30, 0.1), lamb=10))

In [17]:
sigmoid(FFN)

tensor([[1.3888e-11, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00],
        [3.0590e-07, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00],
        [1.0000e+00,

In [18]:
torch.relu(FFN[5][0])

tensor(0., dtype=torch.float64)

In [19]:
sigmoid(FFN[5][0]) 

tensor(1., dtype=torch.float64)

In [20]:
torch.relu(FFN[5][1])

tensor(1., dtype=torch.float64)

In [21]:
sigmoid(FFN[5][1])

tensor(1., dtype=torch.float64)

In [22]:
torch.relu(FFN[5][0]) * sigmoid(FFN[5][0]) + torch.relu(FFN[5][1]) * sigmoid(FFN[5][1])

tensor(1., dtype=torch.float64)

In [23]:
torch.relu(FFN)

tensor([[12.0000,  3.0000,  2.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [11.0000,  2.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 7.3333,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 7.5000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 7.2000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [10.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0

In [24]:
FFN_additional = torch.relu(FFN)*sigmoid(FFN)@ np.array([
    [1.1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
    [1.1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
    [1.1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,],
])
FFN_additional

tensor([[5.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [3.3000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [8.0667, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [8.2500, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [7.9200, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.1000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0736, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [8.8000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
  

In [25]:
output = [itos[i.item()] for i in FFN_additional.sum(dim=-1, dtype=int)]
output

['5', '3', '8', '8', '7', '1', '0', '8']

In [26]:
input[:6]

'65+43='

In [27]:
output[-3:]

['1', '0', '8']

In [28]:
!python addition_TF.py 11+22=033

11+22=033
['3', '3', '3']


In [29]:
# from numpy.linalg import pinv
# from numpy.linalg import inv

# # W_vocab = pinv(W_emb)
# W_vocab = inv(W_emb)

# W_vocab[:, 10:] = 0

# logits = FFN @ W_vocab
# logits

# output = [itos[i.item()] for i in logits.argmax(axis=-1)]
# output